# Load COOP Test Data
## load example filings for the ledger (not complete filings)
downstream integrations (like receipts) wont actually work

### Setup Environment

In [ ]:
# using TEST database
%run /workspaces/lear/tests/data/default-bcr-business-setup-TEST.ipynb

In [ ]:
%run /workspaces/lear/tests/data/common/legal_api_utils_test_data.ipynb

In [ ]:
import copy
import re
import datedelta
from legal_api.core import Filing as CoreFiling
from legal_api.models import Comment, Filing, RegistrationBootstrap
from legal_api.utils.datetime import date, datetime

### Setup the Business Info

In [ ]:
# identifier = 'CP0000842'
# legal_type = identifier[:re.search(r"\d", identifier).start()]
# identifier, legal_type

In [ ]:
# if business := legal_api.models.Business.find_by_identifier(identifier):
#         print(f'{identifier} already exisit in LEAR, DO NOT ATTENP TO LOAD IT AGAIN')
# assert not business, "Business already exists, if you want to build it, set build to True and run the next cell."

In [ ]:
# create_business = True
# if create_business:
#     current_time = datetime.utcnow()
#     business = Business(legal_name=f'legal_name-{identifier}',
#                         founding_date=(current_time - datedelta.datedelta(years=4)),
#                         last_ar_date=current_time,
#                         last_ar_year=current_time.year,
# #                         last_coa_date=current_time,
# #                         last_cod_date=current_time,
#                         last_ledger_timestamp=current_time,
#                         # dissolution_date=EPOCH_DATETIME,
#                         identifier=identifier,
#                         tax_id='BN123456789',
#                         fiscal_year_end_date=current_time,
#                         legal_type=legal_type)
#     business.save()
    
# business.id, business.legal_name, business.founding_date

In [ ]:
bootstrap = RegistrationBootstrap()
bootstrap.identifier = 'T22222212'
bootstrap.account = 668
bootstrap.save()

### Create Incorporation Application

In [ ]:
filing_name = 'incorporationApplication'

filing_date = datetime.utcnow()
filing_month = 0
filing_json = copy.deepcopy(FILING_HEADER)
filing_json['filing'].pop('business')
filing_json['filing']['header']['name'] = filing_name
filing_json['filing'][filing_name] = INCORPORATION
filing_json
filing = factory_pending_IA_filing(bootstrap, filing_json, filing_date, payment_token=None, colin_id=None)

filing_meta = {
    'legalFilings':[filing_name,],
    filing_name:{
}}
meta_data = {**{'applicationDate': filing_date.date().isoformat()}, **filing_meta}

filing._meta_data = meta_data

filing.temp_reg = bootstrap.identifier

filing.save()

In [ ]:
ledger = CoreFiling.ledger(business.id)
ledger_filing = next((f for f in ledger if f.get('filingId')==filing.id), None)

assert ledger_filing['displayName'] == 'Incorporation Application'

In [ ]:
filing.id

In [ ]:
filing_json

In [ ]:
url = "https://dev.oidc.gov.bc.ca/auth/realms/fcf0kpqr/protocol/openid-connect/token"

payload = "grant_type=client_credentials"
basic_hash = base64.b64encode("entity-notebook-service-account:170e88e8-0a57-4d3c-b7f2-770cc2e1456f".encode()).decode()
basic_hash
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Authorization': f'Basic {basic_hash}'
}

response = requests.request("POST", url, headers=headers, data=payload)

token = response.json()['access_token']

In [ ]:
url = "https://auth-api-test.apps.silver.devops.gov.bc.ca/api/v2/entities"

payload = {
  "businessIdentifier": 'T12121212',
  "corpTypeCode": 'CP',
  "name": ""
}

headers = {
  'Authorization': f'Bearer {token}'
}

r = requests.post(url, headers=headers, json=payload)

r.status_code, r.text

In [ ]:
account = 668
url = f'https://auth-api-dev.apps.silver.devops.gov.bc.ca/api/v1/orgs/{account}/affiliations'

payload = {"businessIdentifier": 'T12121212',
           "passCode": ""
          }
headers = {
  'Authorization': f'Bearer {token}'
}

r = requests.post(url, headers=headers, json=payload)

r.status_code, r.text